In [16]:
import pandas as pd 
from data.lib import dormant_data
from backtesting import Backtest, Strategy

In [17]:
X = dormant_data(path='../../../ZALK/Data_sourcing/compressed/BTC-USDT.parquet',start_date ='2022-01-01 00:00:00',
                 end_date='2022-11-01 00:00:00',datz=True)

ori_df = X.copy()
X = X.drop(columns=['quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume'])

X['date'] = pd.to_datetime(X['date'])
X = X.set_index('date')
X.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
X.head()

              
    ---> Does df have NaN values ? == False
    ---> Total Rows In The Data  ? == 437760
            


,Open,High,Low,Close,Volume
date,,,,,
2022-01-01 00:01:00,46250.000000,46344.230469,46234.390625,46312.761719,42.381062
2022-01-01 00:02:00,46312.761719,46381.691406,46292.750000,46368.730469,51.299549
2022-01-01 00:03:00,46368.730469,46391.488281,46314.261719,46331.078125,30.458941
2022-01-01 00:04:00,46331.070312,46336.101562,46300.000000,46321.339844,20.960291
2022-01-01 00:05:00,46321.339844,46443.558594,46280.000000,46436.031250,35.866821


In [18]:
## Volatility based approach using ATR
## Moving average trailing stops
## Traditional stop/limit (using support and resistance)
## Increasing or decreasing the time horizon limit(last step)

## Problem with the above steps --->
## No way to find sl and tp boundary like with fixed sl/tp

def I_helper(series): return series

class Boltning(Strategy):
    
    def init(self):
        self.time_limit = 720 ## 12 hours

    def next(self):

        if self.trades:
            last_time  = max(0, self.trades[-1].entry_bar-self.time_limit)
            for trades in self.trades:
                if trades.entry_bar <= last_time:
                   trades.close(portion=1)

        price = self.data.Close[-1]
        self.buy(size=1,sl=0.96*price, tp=1.04*price)

In [19]:
## Make sure the sl/tp matches from the stategy file....
bt = Backtest(X, Boltning, cash=100000000, commission=.002)
stats = bt.run()

pd.DataFrame(stats['_trades']).to_csv('data/trades.csv', index=False)
print(stats)
#bt.plot()

In [14]:
## Remenber to subtract 1 from the digit used in the strategy... 
sl = -0.035
tp =  0.035

results = pd.read_csv('data/trades.csv')
results = results.drop(columns=['PnL','ExitTime','ExitPrice','EntryPrice','Size'])
df = results.copy()

results['sl'] = (df['ReturnPct'].where((df['ReturnPct'] < sl))).fillna(0)
results['tp'] = (df['ReturnPct'].where((df['ReturnPct'] > tp))).fillna(0)

results.loc[results['sl'] < sl, 'sl'] = -1
results.loc[results['tp'] > tp, 'tp'] =  1

results['labels'] = results['sl'] + results['tp']
results = results.drop(columns=['sl','tp'])

results = results.sort_values('EntryBar').reset_index(drop=True)
results

,EntryBar,ExitBar,ReturnPct,EntryTime,Duration,labels
0,2,723,0.006967,2021-01-01 00:03:00,0 days 12:01:00,0.0
1,3,724,0.007152,2021-01-01 00:04:00,0 days 12:01:00,0.0
2,4,725,0.006444,2021-01-01 00:05:00,0 days 12:01:00,0.0
3,5,726,0.007695,2021-01-01 00:06:00,0 days 12:01:00,0.0
4,6,727,0.007510,2021-01-01 00:07:00,0 days 12:01:00,0.0
...,...,...,...,...,...,...
479957,479959,479963,-0.002520,2021-11-30 23:56:00,0 days 00:04:00,0.0
479958,479960,479963,-0.002862,2021-11-30 23:57:00,0 days 00:03:00,0.0
479959,479961,479963,-0.002738,2021-11-30 23:58:00,0 days 00:02:00,0.0
479960,479962,479963,-0.002713,2021-11-30 23:59:00,0 days 00:01:00,0.0


In [15]:
## Merge the labels and the original data

## 720 is value of the time horizion set in the Boltning strategy 
ori_df  =  ori_df.iloc[2 :-720]
results = results.iloc[  :-720]

ori_df = ori_df.reset_index(drop=True)
print(ori_df.shape, results.shape)

ori_df['labels'] = results['labels'].copy()
ori_df.to_csv('data/result.csv', index=False)

(478520, 11) (479242, 6)
